In [1]:
# !pip install deap
# uncomment and run the first time as most probably deap library may not be already installed

In [6]:
!pip install deap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.8 MB/s eta 0:00:00


In [7]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# -----------------------------------
# STEP 1: Problem Definition
# -----------------------------------
# We need to optimize a delivery route by minimizing total travel distance.
# The warehouse is at index 0, and we must visit all locations exactly once.

# Distance matrix representing the travel cost between locations
distance_matrix = np.array([
    [0, 10, 15, 20, 25, 30],  # Warehouse
    [10, 0, 35, 25, 30, 20],  # Location 1
    [15, 35, 0, 30, 20, 25],  # Location 2
    [20, 25, 30, 0, 15, 10],  # Location 3
    [25, 30, 20, 15, 0, 35],  # Location 4
    [30, 20, 25, 10, 35, 0]   # Location 5
])

num_locations = len(distance_matrix)  # Total locations (including warehouse)
population_size = 10   # Number of candidate solutions in each generation
generations = 100      # Number of iterations
mutation_rate = 0.2    # Probability of mutation

# -----------------------------------
# STEP 2: Genetic Algorithm Setup
# -----------------------------------
# We define a DEAP "FitnessMin" class to represent our objective (minimize distance).
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize distance
creator.create("Individual", list, fitness=creator.FitnessMin)  # A delivery route is a list of locations

toolbox = base.Toolbox()

# Function to generate a valid random route (excluding warehouse)
def create_valid_route():
    """Creates a shuffled list of locations excluding the warehouse (index 0)."""
    route = list(range(1, num_locations))  # Locations 1 to num_locations-1
    random.shuffle(route)  # Shuffle locations randomly
    return creator.Individual(route)  # Return as DEAP Individual

toolbox.register("individual", tools.initIterate, creator.Individual, create_valid_route)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# -----------------------------------
# STEP 3: Fitness Function (Objective Function)
# -----------------------------------
# The fitness function calculates the total travel distance for a given route.
def route_distance(individual):
    """Computes total travel distance for a given route."""
    route = [0] + individual + [0]  # Start and end at the warehouse
    total_distance = sum(distance_matrix[route[i], route[i + 1]] for i in range(len(route) - 1))
    return total_distance,  # Return as a tuple (DEAP requires tuple format)

toolbox.register("evaluate", route_distance)


# -----------------------------------
# TODO: Extend the Lab (Student Task)
# -----------------------------------
# Modify the fitness function to include time constraints.
# Steps:
# 1. Assume each location has a preferred delivery time slot (e.g., [2, 4, 1, 3, 5]).
# 2. Penalize routes that visit locations later than their required time.
# 3. Modify the `route_distance` function to include a penalty.
# 4. Re-run the genetic algorithm and analyze changes in the best route.


# -----------------------------------
# STEP 4: Genetic Algorithm Operators
# -----------------------------------
# Crossover: Uses Partially Mapped Crossover (PMX) to swap sections between two parents
def custom_pmx(ind1, ind2):
    """Applies PMX crossover after converting to 0-based indices, then converts back."""
    ind1[:] = [x - 1 for x in ind1]  # Convert to 0-based indices
    ind2[:] = [x - 1 for x in ind2]

    tools.cxPartialyMatched(ind1, ind2)  # Apply PMX crossover

    ind1[:] = [x + 1 for x in ind1]  # Convert back to 1-based indices
    ind2[:] = [x + 1 for x in ind2]
    return ind1, ind2  # Return modified individuals

toolbox.register("mate", custom_pmx)  # Crossover function
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=mutation_rate)  # Mutation: Swap elements
toolbox.register("select", tools.selTournament, tournsize=3)  # Selection: Tournament method

# -----------------------------------
# STEP 5: Genetic Algorithm Execution
# -----------------------------------
def genetic_algorithm():
    """Executes the Genetic Algorithm to find the optimal delivery route."""
    population = toolbox.population(n=population_size)  # Create initial population

    # Run the genetic algorithm for a set number of generations
    for generation in range(generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=mutation_rate)  # Apply crossover & mutation
        population = toolbox.select(offspring, k=len(population))  # Select best individuals for next generation

    # Return the best solution found
    best_route = tools.selBest(population, k=1)[0]  # Get best route
    best_distance = route_distance(best_route)[0]  # Compute total distance
    return best_route, best_distance

# Run the genetic algorithm
best_route, best_distance = genetic_algorithm()

# -----------------------------------
# STEP 6: Print Results
# -----------------------------------
print("Best Delivery Route:", [0] + best_route + [0])  # Add warehouse start/end
print("Minimum Distance:", best_distance)


Best Delivery Route: [0, 2, 4, 5, 3, 1, 0]
Minimum Distance: 115


In [8]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# -----------------------------------
# STEP 1: Problem Definition
# -----------------------------------
# Distance matrix representing travel cost between locations
distance_matrix = np.array([
    [0, 10, 15, 20, 25, 30],  # Warehouse
    [10, 0, 35, 25, 30, 20],  # Location 1
    [15, 35, 0, 30, 20, 25],  # Location 2
    [20, 25, 30, 0, 15, 10],  # Location 3
    [25, 30, 20, 15, 0, 35],  # Location 4
    [30, 20, 25, 10, 35, 0]   # Location 5
])

num_locations = len(distance_matrix)  # Total locations (including warehouse)
population_size = 10   # Number of candidate solutions
generations = 100      # Number of iterations
mutation_rate = 0.2    # Probability of mutation

# -----------------------------------
# STEP 2: Genetic Algorithm Setup
# -----------------------------------
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize distance
creator.create("Individual", list, fitness=creator.FitnessMin)  # Delivery route as list

toolbox = base.Toolbox()

# Function to generate a valid random route (excluding warehouse)
def create_valid_route():
    route = list(range(1, num_locations))  # Locations 1 to num_locations-1
    random.shuffle(route)  # Shuffle locations randomly
    return creator.Individual(route)  # Return as DEAP Individual

toolbox.register("individual", tools.initIterate, creator.Individual, create_valid_route)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# -----------------------------------
# STEP 3: Fitness Function (Objective Function) with Time Constraints
# -----------------------------------
# Preferred delivery time slots for each location
preferred_times = [0, 2, 4, 1, 3, 5]  # Warehouse at index 0

def route_distance(individual):
    """Computes total travel distance with penalties for late deliveries."""
    route = [0] + individual + [0]  # Start and end at the warehouse
    total_distance = sum(distance_matrix[route[i], route[i + 1]] for i in range(len(route) - 1))

    penalty = 0
    for i, location in enumerate(individual):
        preferred_time = preferred_times[location]
        if i > preferred_time:  # If visited later than preferred time
            penalty += (i - preferred_time) * 10  # Apply penalty for lateness

    return total_distance + penalty,  # Return as tuple for DEAP compatibility

toolbox.register("evaluate", route_distance)

# -----------------------------------
# STEP 4: Genetic Algorithm Operators
# -----------------------------------
def custom_pmx(ind1, ind2):
    """Applies PMX crossover after converting to 0-based indices."""
    ind1[:] = [x - 1 for x in ind1]  # Convert to 0-based indices
    ind2[:] = [x - 1 for x in ind2]

    tools.cxPartialyMatched(ind1, ind2)  # Apply PMX crossover

    ind1[:] = [x + 1 for x in ind1]  # Convert back to 1-based indices
    ind2[:] = [x + 1 for x in ind2]
    return ind1, ind2  # Return modified individuals

toolbox.register("mate", custom_pmx)  # Crossover function
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=mutation_rate)  # Mutation
toolbox.register("select", tools.selTournament, tournsize=3)  # Selection method

# -----------------------------------
# STEP 5: Genetic Algorithm Execution
# -----------------------------------
def genetic_algorithm():
    """Executes the Genetic Algorithm to find the optimal delivery route."""
    population = toolbox.population(n=population_size)

    for generation in range(generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=mutation_rate)
        population = toolbox.select(offspring, k=len(population))

    best_route = tools.selBest(population, k=1)[0]
    best_distance = route_distance(best_route)[0]
    return best_route, best_distance

# Run the genetic algorithm
best_route, best_distance = genetic_algorithm()

# -----------------------------------
# STEP 6: Print Results
# -----------------------------------
print("Best Delivery Route:", [0] + best_route + [0])  # Add warehouse start/end
print("Minimum Distance:", best_distance)


Best Delivery Route: [0, 2, 3, 1, 4, 5, 0]
Minimum Distance: 165


/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


### Answers to Questions

1. **Before modifying the fitness function, what was the optimal route and distance?**
   - **Optimal Route:** `[0, 2, 4, 5, 3, 1, 0]`
   - **Minimum Distance:** `115`

2. **After including time constraints, how did the route change? Did the cost increase?**
   - **New Optimal Route:** `[0, 2, 3, 1, 4, 5, 0]`
   - **New Minimum Distance:** `165`
   - The route changed from visiting locations 4 and 5 to prioritizing delivery based on preferred time slots. The total distance increased from `115` to `165` due to penalties for not adhering to the preferred delivery times.

3. **What happens when you increase the number of generations? What happens when you update the mutation rate, or change the size of the population? Briefly explain the observations.**
   - **Increasing the Number of Generations:** This allows more exploration of the solution space, potentially leading to better optimization results. However, after a certain point, improvements may diminish as the algorithm converges.
   - **Updating the Mutation Rate:** A higher mutation rate introduces more variability, helping to escape local optima, while a lower rate may lead to premature convergence. The right balance is crucial for effective optimization.
   - **Changing the Size of the Population:** A larger population offers better diversity and exploration, potentially improving results but at the cost of increased computation. A smaller population can converge faster but risks getting stuck in local optima.

These parameters are essential for balancing exploration and exploitation in a genetic algorithm, significantly impacting performance and optimization quality.
